# Employee Profile Exporter
This notebook is created to fulfill the assignment by importing salary data, allowing name-based searches, exporting results, and compressing them into a ZIP file.

## 1. Import Required Libraries

In [1]:
import os
import pandas as pd
import zipfile
import fnmatch
import pickle
from kaggle.api.kaggle_api_extended import KaggleApi

## 2. Load Salary Data from Kaggle

In [2]:
def load_salary_data(download_path="sf_salaries", force_download=False):
    zip_filename = "20112018-salaries-for-san-francisco.zip"
    csv_file = os.path.join(download_path, "Total.csv")
    zip_path = os.path.join(download_path, zip_filename)

    if not force_download and os.path.exists(csv_file):
        print("✅ Total.csv already exists — skipping Kaggle download.\n")
        return pd.read_csv(csv_file, low_memory=False)

    print("🔽 Total.csv not found or re-download requested — downloading from Kaggle...")
    os.makedirs(download_path, exist_ok=True)

    api = KaggleApi()
    api.authenticate()

    dataset_name = "mojtaba142/20112018-salaries-for-san-francisco"
    api.dataset_download_files(dataset_name, path=download_path, unzip=False)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(download_path)

    print("✅ Dataset downloaded and extracted.\n")
    return pd.read_csv(csv_file)

df = load_salary_data()
print(df.head())

🔽 Total.csv not found or re-download requested — downloading from Kaggle...
Dataset URL: https://www.kaggle.com/datasets/mojtaba142/20112018-salaries-for-san-francisco
✅ Dataset downloaded and extracted.

        EmployeeName                                        JobTitle  \
0     NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY   
1       GARY JIMENEZ                 CAPTAIN III (POLICE DEPARTMENT)   
2     ALBERT PARDINI                 CAPTAIN III (POLICE DEPARTMENT)   
3  CHRISTOPHER CHONG            WIRE ROPE CABLE MAINTENANCE MECHANIC   
4    PATRICK GARDNER    DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)   

     BasePay OvertimePay   OtherPay      Benefits   TotalPay  \
0  167411.18         0.0  400184.25  Not Provided  567595.43   
1  155966.02   245131.88  137811.38  Not Provided  538909.28   
2  212739.13   106088.18    16452.6  Not Provided  335279.91   
3    77916.0    56120.71   198306.9  Not Provided  332343.61   
4   134401.6      9737.0  182234.59  Not P

C:\Users\oyd21\AppData\Local\Temp\ipykernel_39652\2086861892.py:23: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(csv_file)


## 3. Build Dictionary for Fast Name-Based Lookup

In [3]:
def get_employee_dict(df, cache_file="employee_dict.pkl", force_refresh=False):
    def normalize_keys(d):
        return {k.upper(): v for k, v in d.items()}

    if not force_refresh and os.path.exists(cache_file):
        with open(cache_file, "rb") as f:
            print("✅ Loaded cached employee dictionary.\n")
            cached_dict = pickle.load(f)
            return normalize_keys(cached_dict)

    print("🔄 Building employee dictionary...")
    grouped = df.groupby('EmployeeName').apply(lambda x: x.to_dict(orient='records')).to_dict()
    employee_dict = normalize_keys(grouped)

    with open(cache_file, "wb") as f:
        pickle.dump(employee_dict, f)
        print("✅ Dictionary cached to disk.\n")

    return employee_dict

employee_dict = get_employee_dict(df, cache_file=os.path.join("sf_salaries", "employee_dict.pkl"))

🔄 Building employee dictionary...


C:\Users\oyd21\AppData\Local\Temp\ipykernel_39652\1819501364.py:12: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped = df.groupby('EmployeeName').apply(lambda x: x.to_dict(orient='records')).to_dict()


✅ Dictionary cached to disk.



## 4. Lookup and Export Functions

In [4]:
def get_employee_details(name):
    try:
        details = employee_dict.get(name.upper())
        if details is None:
            return f"No records found for employee: {name}"
        return details
    except Exception as e:
        return f"Error retrieving employee details: {e}"

def export_employee_profile(name, output_folder="Employee Profile", verbose=True):
    details = get_employee_details(name)

    if isinstance(details, str):
        if verbose:
            print(details)
        return

    os.makedirs(output_folder, exist_ok=True)
    filename = f"{name.replace(' ', '_')}.csv"
    filepath = os.path.join(output_folder, filename)

    pd.DataFrame(details).to_csv(filepath, index=False)

## 5. Zip Exported Files

In [5]:
def zip_exported_profiles(folder="Employee Profile", zip_name="Employee Profiles.zip"):
    count = 0
    with zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder):
            for file in files:
                if file.endswith(".csv"):
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=file)
                    count += 1
    print(f"✅ {count} profile(s) zipped into: {zip_name}")

## 6. Clear Old Exports (Optional Cleanup)

In [6]:
def clear_export_folder(folder="Employee Profile"):
    if os.path.exists(folder):
        for file in os.listdir(folder):
            file_path = os.path.join(folder, file)
            if os.path.isfile(file_path) and file.endswith(".csv"):
                os.remove(file_path)

## 7. Sample Test Case

In [7]:
clear_export_folder()
export_employee_profile("RAY CRAWFORD")
zip_exported_profiles()

✅ 1 profile(s) zipped into: Employee Profiles.zip


## 8. Print names in pages of 20 for easier reading

In [8]:
def display_paginated(names, page_size=20):
    total = len(names)
    for i in range(0, total, page_size):
        batch = names[i:i + page_size]
        for name in batch:
            print(f"→ {name}")
        if i + page_size < total:
            input("\nPress Enter to see the next 20 records...")
        else:
            print("✅ End of list.\n")

## 9. Interactive prompt to search and export employee data

In [9]:
if __name__ == "__main__":
    print("📄 Employee Profile Exporter")
    print("Enter an employee name using * as a wildcard (e.g., 'JOHN*' or '*FORD').")
    print("You must include at least 3 non-wildcard characters.")
    print("Type 'exit' to quit.\n")

    while True:
        pattern = input("Employee Name Pattern: ").strip()

        if pattern.lower() == 'exit':
            print("Goodbye!")
            break

        if len(pattern.replace('*', '')) < 3:
            print("⚠️ Please enter at least 3 non-wildcard characters.")
            continue

        matched_names = fnmatch.filter(df['EmployeeName'].str.upper(), pattern.upper())

        if not matched_names:
            print(f"❌ No employee names matched the pattern '{pattern}'.")
            continue

        print(f"\n🔍 Found {len(matched_names)} matching record(s).")
        print(f"🧾 {len(set(matched_names))} unique employee name(s) will be exported.\n")

        view = input("Would you like to view them before exporting? (y/n): ").strip().lower()
        suppress_export_print = (view != 'y')

        if view == 'y':
            view_mode = input("Show all at once or in batches of 20? (all/20): ").strip().lower()
            if view_mode == 'all':
                for name in matched_names:
                    print(f"→ {name}")
            elif view_mode == '20':
                display_paginated(matched_names)
            else:
                print("⚠️ Unrecognized option. Skipping display.\n")
        elif view == 'n':
            print("⏭️ Skipping preview.\n")
        else:
            print("⚠️ Invalid input. Skipping display.\n")

        print("📦 Starting export of matched profiles...\n")
        clear_export_folder()
        for name in matched_names:
            export_employee_profile(name, verbose=False)
        zip_exported_profiles()

        again = input("\nWould you like to search again? (y/n): ").strip().lower()
        if again != 'y':
            print("Goodbye!")
            break


📄 Employee Profile Exporter
Enter an employee name using * as a wildcard (e.g., 'JOHN*' or '*FORD').
You must include at least 3 non-wildcard characters.
Type 'exit' to quit.


🔍 Found 119 matching record(s).
🧾 43 unique employee name(s) will be exported.

⏭️ Skipping preview.

📦 Starting export of matched profiles...

✅ 43 profile(s) zipped into: Employee Profiles.zip
Goodbye!
